In [1]:
# Скачиваем данные

# papaver_all_with_tags
!wget https://drive.google.com/open?id=1muuQI5SdgqPui3nCYzkO8AVwChjHf6A4
  
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#link = 'https://drive.google.com/open?id=1DPZZQ43w8brRhbEMolgLqOWKbZbE-IQu' # The shareable link

downloaded = drive.CreateFile({'id':'1muuQI5SdgqPui3nCYzkO8AVwChjHf6A4'}) 
downloaded.GetContentFile('hg.zip')  

--2019-04-03 16:21:28--  https://drive.google.com/open?id=1muuQI5SdgqPui3nCYzkO8AVwChjHf6A4
Resolving drive.google.com (drive.google.com)... 74.125.141.113, 74.125.141.139, 74.125.141.101, ...
Connecting to drive.google.com (drive.google.com)|74.125.141.113|:443... connected.
HTTP request sent, awaiting response... 307 OK
Location: https://drive.google.com/file/d/1muuQI5SdgqPui3nCYzkO8AVwChjHf6A4/view?usp=drive_open [following]
--2019-04-03 16:21:28--  https://drive.google.com/file/d/1muuQI5SdgqPui3nCYzkO8AVwChjHf6A4/view?usp=drive_open
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘open?id=1muuQI5SdgqPui3nCYzkO8AVwChjHf6A4’

open?id=1muuQI5Sdgq     [ <=>                ] 131.23K  --.-KB/s    in 0.008s  

2019-04-03 16:21:28 (15.4 MB/s) - ‘open?id=1muuQI5SdgqPui3nCYzkO8AVwChjHf6A4’ saved [134381]

    100% |████████████████████████████████| 993kB 20.1MB/s 


In [0]:
import zipfile
zip_ref = zipfile.ZipFile('hg.zip', 'r')
zip_ref.extractall('files')
zip_ref.close()

In [0]:
# Ячейка на всякий случай
#!pip install nltk
#!pip install pymorphy2[fast]
#!pip install pandas

In [0]:
import nltk
import pandas as pd
import os, re

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
df = pd.DataFrame()

In [0]:
# Функция для чтения датасета

def read_files(path):
    files = []
    for i in os.listdir(path):
        if "txt" in i:
            files.append(i)
    texts = []
    for i in files:
        with open(path+"/"+i, "r") as f:
            text = ""
            for line in f.readlines():
                if line.strip():
                    text += line + "\n"
            texts.append(text)
    return texts 

In [0]:
data = read_files('files')
df['raw_texts'] = data

In [0]:
def find_main_part(text): 
    lines = [line for line in text.split('\n')]
    beg, end = 0, 0 
    for num, line in enumerate(lines): 
      # lines[num] = re.sub(r'\\[a-z]+', '', line)
      if 'установил' in line.lower() or 'у с т а н о в и л' in line.lower(): 
        beg = num + 1
      if 'приговорил' in line.lower() or 'п р и г о в о р и л' in line.lower(): 
        end = num + 2
      if beg and end: 
          main_part = [stroka.strip() for stroka in lines[beg:end]]
          return ' '.join(main_part)

In [0]:
df['raw_main_part'] = df['raw_texts'].apply(find_main_part)

In [10]:
! pip install pymorphy2[fast]

    100% |████████████████████████████████| 51kB 2.0MB/s 
    100% |████████████████████████████████| 7.1MB 3.3MB/s 
    100% |████████████████████████████████| 256kB 28.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/d4/88/d0/4e4abc83eb8f59a71e8dbd8ba99fd5615a3af1fac1ef7f8825
Successfully built DAWG


In [11]:
# Функция для разбиения текста на абзацы по тегам
# При этом куски-юридизмы (= текст между тегами \jurb и \jure) заменить на JUR или что-то подобное
beg_tag = re.compile(r"\\\w+b\b")
end_tag = re.compile(r"\\\w+e\b")
tag = re.compile(r"\\\w+[e|b]\b")
jur_tag = re.compile(r"\\jur[e|b]\b")


#Заменяем всё внутри тегов на JUR
def no_jur(text):
    jur_split = re.split(jur_tag, text)
    text_without_jur = ""
    for i in jur_split:
        if i in (jur_split[1::2]):
            #Не понимаю, почему при этом появляется второй бэкслеш"
            text_without_jur += "\jurb JUR \jure"
        else:
            text_without_jur += i
    return text_without_jur

no_jur("Это имитирует \jurb юридизмы всякие \jure, от которых нам надо \jurb юридически \jure избавляться")

'Это имитирует \\jurb JUR \\jure, от которых нам надо \\jurb JUR \\jure избавляться'

In [0]:
def del_tags(text):
    pattern = re.compile(r'\\[a-z]+')
    # new_text = re.sub(pattern, '', text)
    # Небольшая правка: привела текст к строке -- иначе выдавало ошибку (Настя)
    new_text = re.sub(pattern, '', str(text))
    return new_text

In [0]:
df['main_part_no_tags'] = df['raw_main_part'].apply(del_tags)

In [14]:
# Функция для очистки датасета от стоп-слов и лемматизации
from string import punctuation
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    if text:
      words = [word.strip(punct) for word in text.lower().split()] 
      words = [word for word in words if word and word not in stops]
      words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

      return words

normalize("Господи, почему я такая тупая и не нахожу в себе силы даже написать поганые регулярки, пока более успешные коллеги уже сдают статью в середине апреля.")

['господь',
 'почему',
 'такой',
 'тупой',
 'находить',
 'сила',
 'написать',
 'поганый',
 'регулярка',
 'пока',
 'успешный',
 'коллега',
 'сдавать',
 'статья',
 'середина',
 'апрель']

In [0]:
df['tokens_main_part'] = df['main_part_no_tags'].apply(normalize)

In [16]:
#Собственно разбиваем
def paragraphs_split(text): 
    return [par.strip() for par in re.split(tag, text) if par.strip()]
    
paragraphs_split(no_jur(r"Эта регулярка просто разбивает \jurb юридический \jure \textb текст \texte на абзацы по тегам, но при этом никак не подписывает их \nameb названия \namee. \mehb Это затрудняет работу, но улучшать я пока что-то не готова. \mehe"))

['Эта регулярка просто разбивает',
 'JUR',
 'текст',
 'на абзацы по тегам, но при этом никак не подписывает их',
 'названия',
 '.',
 'Это затрудняет работу, но улучшать я пока что-то не готова.']

In [0]:
def split_no_jur(text):
  return paragraphs_split(no_jur(str(text)))

In [0]:
# Сегментируем на предложения

from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer

train_data = df['raw_main_part'].dropna()
trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
trainer.train('\n'.join(train_data))
 
tokenizer = PunktSentenceTokenizer(trainer.get_params())
  

In [0]:
def segmentate(text):
  return tokenizer.tokenize(text)

In [0]:
df['sentences'] = df['main_part_no_tags'].apply(segmentate)

In [21]:
# Посмотрим, что он там насегментировал
for num, i in enumerate(df['sentences'][:3]):
  for sent in i:
    print(sent, '\n')

  Подсудимый С.Н.Ю. совершил два преступления незаконных изготовления и хранения без цели сбыта наркотических средств, в крупном размере.  Преступления подсудимым совершены при следующих обстоятельствах. 

С.Н.Ю., ДД.ММ.ГГГГ, примерно в 18 часов 00 минут, находясь в помещении кухни <адрес>, имея умысел на незаконные изготовление и хранение без цели сбыта наркотического средства, с целью последующего личного употребления, действуя умышленно, путем экстракции семян кондитерского мака и растворителя, с последующим выпариванием на огне, незаконно без цели сбыта, изготовил кустарное наркотическое средство – экстракт маковой соломы, точной массой на момент исследования соответственно:1,1 грамма и 0,17 грамма, общей массой в сухом виде 1,27 грамма, являющееся согласно Постановления Правительства РФ за № 76 от 07 февраля 2006 года «Об утверждении крупного и особо крупного размеров наркотических средств и психотропных веществ, а также крупного и особо крупного размеров для растений, содержащих 

In [0]:
tokenizer._params.abbrev_types.add('гр.')

In [0]:
df['sentences'] = df['main_part_no_tags'].apply(segmentate)

In [24]:
# Посмотрим, что теперь он насегментировал
for num, i in enumerate(df['sentences'][:3]):
  for sent in i:
    print(sent, '\n')

  Подсудимый С.Н.Ю. совершил два преступления незаконных изготовления и хранения без цели сбыта наркотических средств, в крупном размере.  Преступления подсудимым совершены при следующих обстоятельствах. 

С.Н.Ю., ДД.ММ.ГГГГ, примерно в 18 часов 00 минут, находясь в помещении кухни <адрес>, имея умысел на незаконные изготовление и хранение без цели сбыта наркотического средства, с целью последующего личного употребления, действуя умышленно, путем экстракции семян кондитерского мака и растворителя, с последующим выпариванием на огне, незаконно без цели сбыта, изготовил кустарное наркотическое средство – экстракт маковой соломы, точной массой на момент исследования соответственно:1,1 грамма и 0,17 грамма, общей массой в сухом виде 1,27 грамма, являющееся согласно Постановления Правительства РФ за № 76 от 07 февраля 2006 года «Об утверждении крупного и особо крупного размеров наркотических средств и психотропных веществ, а также крупного и особо крупного размеров для растений, содержащих 

На большем количестве данных могут быть другие результаты, так что еще нужно  будет докручивать.

In [25]:
df.head()

,raw_texts,raw_main_part,main_part_no_tags,tokens_main_part,sentences
0,﻿Дело № 1-37/2012 г.\n\nП Р И Г О В О Р\n\n\in...,\fabb Подсудимый С.Н.Ю. совершил два преступл...,Подсудимый С.Н.Ю. совершил два преступления ...,"[подсудимый, с.н.ть, совершить, преступление, ...",[ Подсудимый С.Н.Ю. совершил два преступления...
1,﻿Копия\n\nДело № 1-141\n\nП Р И Г О В О Р\n\n\...,\fabb Шериева И.Г. незаконно изготовила и хра...,Шериева И.Г. незаконно изготовила и хранила ...,"[шериева, и.г, незаконно, изготовить, хранить,...",[ Шериева И.Г. незаконно изготовила и хранила...
2,Уголовное дело №1-504/2012\n\nПРИГОВОР\n\nИмен...,\fabb ФИО1 имея преступный умысел на незакон...,ФИО1 имея преступный умысел на незаконный с...,"[фио1, иметь, преступный, умысел, незаконный, ...",[ ФИО1 имея преступный умысел на незаконный ...
3,Дело № 1-107/2012 ПРИГОВОР Именем Российской Ф...,\fabb Усатов С.В. содержал притон для потребл...,Усатов С.В. содержал притон для потребления ...,"[усат, с.в, содержимый, притон, потребление, н...",[ Усатов С.В. содержал притон для потребления...
4,К делу № 1-444/10\n\nПРИГОВОР\n\nИменем Россий...,\fabb В.Н. Лобиян совершил незаконное изготов...,В.Н. Лобиян совершил незаконное изготовление...,"[в.н, лобиян, совершить, незаконный, изготовле...",[ В.Н. Лобиян совершил незаконное изготовлени...


## Как ещё достать юридизмы? Пробуем N-грамы!

In [0]:
import gensim
import itertools
from pymorphy2 import MorphAnalyzer
from string import punctuation
morph = MorphAnalyzer()

In [0]:
#train
ph = gensim.models.Phrases(df['tokens_main_part'])

In [28]:
ph2 = gensim.models.Phrases(ph[df['tokens_main_part']])
p2 = gensim.models.phrases.Phraser(ph2)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [30]:
p2[df['tokens_main_part'][0]][:50]

['подсудимый_с.н.ть',
 'совершить_преступление',
 'незаконный_изготовление',
 'хранение_цель',
 'сбыт',
 'наркотический_средство',
 'крупный_размер',
 'преступление',
 'подсудимый',
 'совершить_следующий',
 'обстоятельство',
 'с.н.ть',
 'дд.мм.гггга',
 'примерно_18',
 'часы_00',
 'минута',
 'находиться',
 'помещение_кухня',
 'адрес',
 'иметь_умысел',
 'незаконный_изготовление',
 'хранение_цель',
 'сбыт',
 'наркотический_средство',
 'цель_последующий',
 'личный_употребление',
 'действовать_умышленно',
 'путём_экстракция',
 'семя_кондитерский',
 'мак_растворитель',
 'последующий_выпаривание',
 'огонь_незаконно',
 'цель_сбыт',
 'изготовить_кустарный',
 'наркотический_средство',
 'экстракт_маковый',
 'солома',
 'точный',
 'масса',
 'момент_исследование',
 'соответственно:1,1',
 'грамм',
 '0,17',
 'грамм',
 'общий_масса',
 'сухой_вид',
 '1,27',
 'грамм',
 'являться',
 'согласно_постановление']

In [0]:
def split_txt(text):
    txt = []
    for sent in text:
      for word in sent.split(' '):
        word = word.strip(punct).lower()
        if word:
          txt.append(word)
    return txt

df['sentences_spl'] = df['sentences'].apply(split_txt)

In [0]:
# Глумливые регулярки для юридизмов

jur = re.compile("(ук рф|ук рос|упк рос|упк рф|постановлени.{1,4} правительств|постановлени.{1,4} пленум|федеральн.{2,3} закон|постановлени.{1,4} государственной дум|спис.{2,3}|переч.{2,4}|уголовно-процесс|уголовн.{2,4} кодекс|конституц)") 

sentences = ["В соответствии с ч. 1 ст. 314 УПК РФ, обвиняемый вправе при наличии согласия государственного обвинителя и потерпевшего, заявить о согласии с предъявленным ему обвинением и ходатайствовать о постановлении приговора без проведения судебного разбирательства в общем порядке по уголовным делам о преступлениях, наказание за которые, предусмотренное УК РФ, не превышает десяти лет лишения свободы.",
            "Экстракт маковой соломы (концентрат маковой соломы) включен в список № 1 «Перечня наркотических средств, психотропных веществ и их прекурсоров, подлежащих контролю в РФ», утвержденного постановлением Правительства РФ № 1002 от 01 октября 2012 года, и относится к наркотическим средствам, оборот которых в Российской Федерации запрещен в соответствии с законодательством Российской Федерации международными договорами Российской Федерации, а наркотическое средство - экстракт маковой соломы общим весом 1, 67 грамма относит к значительному размеру.",
            "В соответствие с п. 1 ч. 8 ст. 246 УПК РФ, государственный обвинитель до удаления суда в совещательную комнату для постановления приговора может изменить обвинение в сторону смягчения путем исключения из юридической квалификации деяния признаков преступления, отягчающих наказание.",
            "В этом предложении нет ничего, что соответствует нашим чудесным регуляркам",
            "Согласно Постановления Правительства Российской Федерации №76 от 07.02.06 г. «Об утверждении крупного и особо крупного размеров наркотических средств и психотропных веществ для целей статей 228, 228.1 и 229 Уголовного кодекса Российской Федерации» (в редакции Постановления Правительства Российской Федерации от 30.10.2010 года № 882), особо крупным размером признается масса каннабис (марихуаны) свыше 100 грамм."]

In [0]:
def reg_jur(sentences):
    juridisms = []
    for sent in sentences: 
        if (re.findall(jur, sent.lower())):
            # sent = 'JUR'
          juridisms.append(sent)
    return juridisms

In [0]:
## ВОПЛЬ ОТЧАЯНИЯ

def replace_jur(sentences):
  sent_bez_jur = []
  for sentence in sentences:
    if '\jurb' in sentence:
      if '\jure' in sentence:
        jur = sentence[sentence.index('\jurb'):sentence.index('\jure')+5]
      else:
        jur = sentence[sentence.index('\jurb'):]
      sentence = sentence.replace(jur, 'JUR')
    if '\jure' in sentence and '\jurb' not in sentence:
      jur = sentence[:sentence.index('\jure')+5]
      sentence = sentence.replace(jur, 'JUR')
    sent_bez_jur.append(sentence)
  return sent_bez_jur

In [0]:
sent = []
for text in df['raw_main_part']:
  sent.append(segmentate(str(text)))

df['sentences_with_tags'] = sent

In [0]:
df['sentences_bez_jur'] = df['sentences_with_tags'].apply(replace_jur)

In [0]:
df['paragraphs'] = df['raw_main_part'].apply(split_no_jur)

In [0]:
df['paragraphs_tokens'] = df['paragraphs'].apply(split_txt)

In [67]:
df['paragraphs'][0]

['Подсудимый С.Н.Ю. совершил два преступления незаконных изготовления и хранения без цели сбыта наркотических средств, в крупном размере.  Преступления подсудимым совершены при следующих обстоятельствах.  С.Н.Ю., ДД.ММ.ГГГГ, примерно в 18 часов 00 минут, находясь в помещении кухни <адрес>, имея умысел на незаконные изготовление и хранение без цели сбыта наркотического средства, с целью последующего личного употребления, действуя умышленно, путем экстракции семян кондитерского мака и растворителя, с последующим выпариванием на огне, незаконно без цели сбыта, изготовил кустарное наркотическое средство – экстракт маковой соломы, точной массой на момент исследования соответственно:1,1 грамма и 0,17 грамма, общей массой в сухом виде 1,27 грамма, являющееся согласно Постановления Правительства РФ за № 76 от 07 февраля 2006 года «Об утверждении крупного и особо крупного размеров наркотических средств и психотропных веществ, а также крупного и особо крупного размеров для растений, содержащих н

In [91]:
df['sents_with_jur_bez_tags'][0]

"[' \\ Подсудимый С.Н.Ю. совершил два преступления незаконных изготовления и хранения без цели сбыта наркотических средств, в крупном размере.  Преступления подсудимым совершены при следующих обстоятельствах.', 'С.Н.Ю., ДД.ММ.ГГГГ, примерно в 18 часов 00 минут, находясь в помещении кухни <адрес>, имея умысел на незаконные изготовление и хранение без цели сбыта наркотического средства, с целью последующего личного употребления, действуя умышленно, путем экстракции семян кондитерского мака и растворителя, с последующим выпариванием на огне, незаконно без цели сбыта, изготовил кустарное наркотическое средство – экстракт маковой соломы, точной массой на момент исследования соответственно:1,1 грамма и 0,17 грамма, общей массой в сухом виде 1,27 грамма, являющееся согласно Постановления Правительства РФ за № 76 от 07 февраля 2006 года «Об утверждении крупного и особо крупного размеров наркотических средств и психотропных веществ, а также крупного и особо крупного размеров для растений, содер

In [40]:
df.head()

,raw_texts,raw_main_part,main_part_no_tags,tokens_main_part,sentences,sentences_spl,sentences_with_tags,sentences_bez_jur,paragraphs,paragraphs_tokens
0,﻿Дело № 1-37/2012 г.\n\nП Р И Г О В О Р\n\n\in...,\fabb Подсудимый С.Н.Ю. совершил два преступл...,Подсудимый С.Н.Ю. совершил два преступления ...,"[подсудимый, с.н.ть, совершить, преступление, ...",[ Подсудимый С.Н.Ю. совершил два преступления...,"[подсудимый, с.н.ю, совершил, два, преступлени...",[ \fabb Подсудимый С.Н.Ю. совершил два преступ...,[ \fabb Подсудимый С.Н.Ю. совершил два преступ...,[Подсудимый С.Н.Ю. совершил два преступления н...,"[подсудимый, с.н.ю, совершил, два, преступлени..."
1,﻿Копия\n\nДело № 1-141\n\nП Р И Г О В О Р\n\n\...,\fabb Шериева И.Г. незаконно изготовила и хра...,Шериева И.Г. незаконно изготовила и хранила ...,"[шериева, и.г, незаконно, изготовить, хранить,...",[ Шериева И.Г. незаконно изготовила и хранила...,"[шериева, и.г, незаконно, изготовила, и, храни...",[ \fabb Шериева И.Г. незаконно изготовила и хр...,[ \fabb Шериева И.Г. незаконно изготовила и хр...,[Шериева И.Г. незаконно изготовила и хранила б...,"[шериева, и.г, незаконно, изготовила, и, храни..."
2,Уголовное дело №1-504/2012\n\nПРИГОВОР\n\nИмен...,\fabb ФИО1 имея преступный умысел на незакон...,ФИО1 имея преступный умысел на незаконный с...,"[фио1, иметь, преступный, умысел, незаконный, ...",[ ФИО1 имея преступный умысел на незаконный ...,"[фио1, имея, преступный, умысел, на, незаконны...",[ \fabb ФИО1 имея преступный умысел на незако...,[ \fabb ФИО1 имея преступный умысел на незако...,[ФИО1 имея преступный умысел на незаконный сбы...,"[фио1, имея, преступный, умысел, на, незаконны..."
3,Дело № 1-107/2012 ПРИГОВОР Именем Российской Ф...,\fabb Усатов С.В. содержал притон для потребл...,Усатов С.В. содержал притон для потребления ...,"[усат, с.в, содержимый, притон, потребление, н...",[ Усатов С.В. содержал притон для потребления...,"[усатов, с.в, содержал, притон, для, потреблен...",[ \fabb Усатов С.В. содержал притон для потреб...,[ \fabb Усатов С.В. содержал притон для потреб...,[Усатов С.В. содержал притон для потребления н...,"[усатов, с.в, содержал, притон, для, потреблен..."
4,К делу № 1-444/10\n\nПРИГОВОР\n\nИменем Россий...,\fabb В.Н. Лобиян совершил незаконное изготов...,В.Н. Лобиян совершил незаконное изготовление...,"[в.н, лобиян, совершить, незаконный, изготовле...",[ В.Н. Лобиян совершил незаконное изготовлени...,"[в.н, лобиян, совершил, незаконное, изготовлен...",[ \fabb В.Н. Лобиян совершил незаконное изгото...,[ \fabb В.Н. Лобиян совершил незаконное изгото...,"[В.Н. Лобиян совершил незаконное изготовление,...","[в.н, лобиян, совершил, незаконное, изготовлен..."


In [41]:
#надо скачать файл по этой ссылке: https://drive.google.com/open?id=1fXzzKUWmSiCnzNuW8HmxuoXWmaGDoJqv
!wget https://drive.google.com/open?id=1fXzzKUWmSiCnzNuW8HmxuoXWmaGDoJqv
  
  
 # Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id':'1fXzzKUWmSiCnzNuW8HmxuoXWmaGDoJqv'}) 
downloaded.GetContentFile('jur.txt')  

--2019-04-03 16:37:49--  https://drive.google.com/open?id=1fXzzKUWmSiCnzNuW8HmxuoXWmaGDoJqv
Resolving drive.google.com (drive.google.com)... 74.125.141.139, 74.125.141.101, 74.125.141.102, ...
Connecting to drive.google.com (drive.google.com)|74.125.141.139|:443... connected.
HTTP request sent, awaiting response... 307 OK
Location: https://drive.google.com/file/d/1fXzzKUWmSiCnzNuW8HmxuoXWmaGDoJqv/view?usp=drive_open [following]
--2019-04-03 16:37:49--  https://drive.google.com/file/d/1fXzzKUWmSiCnzNuW8HmxuoXWmaGDoJqv/view?usp=drive_open
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘open?id=1fXzzKUWmSiCnzNuW8HmxuoXWmaGDoJqv’

open?id=1fXzzKUWmSi     [ <=>                ] 132.00K  --.-KB/s    in 0.01s   

2019-04-03 16:37:49 (9.65 MB/s) - ‘open?id=1fXzzKUWmSiCnzNuW8HmxuoXWmaGDoJqv’ saved [135165]



In [0]:
jur_lines = []

with open("jur.txt", "r") as f:
    for line in f.readlines():
        if len(line.strip()) > 10:
            jur_lines.append(line)
            

In [43]:
#и тут уже смотрим на полноту
 
len (jur_lines), len(reg_jur(jur_lines))

(319, 308)

In [0]:
#Проверяем, что из "эталона" не попало
def not_reg_jur(sentences):
    not_juridisms = []
    for sent in sentences: 
        if (re.findall(jur, sent.lower())) == []:  
            not_juridisms.append(sent)
    return not_juridisms

In [45]:
not_reg_jur(jur_lines)

['Иванов М.С. осознает характер и последствия заявленного им ходатайства. Данное ходатайство заявлено им добровольно и после проведения консультации с защитником.\n',
 'за оконченное преступление - от максимального срока наиболее строгого вида наказания, предусмотренного за совершенное преступление санкцией соответствующей статьи;\n',
 'По смыслу закона в пользу подсудимого толкуются не только неустранимые сомнения в его виновности в целом, но и неустранимые сомнения, касающиеся отдельных эпизодов предъявленного обвинения, формы вины, степени и характера участия в совершении преступления и.т.д. \n',
 'Обвинительный приговор постановляется лишь при условии, если в ходе судебного разбирательства виновность подсудимого в совершении преступления доказана.  \\jure\n',
 'Ни одно из доказательств, не имеет заранее установленной силы, а каждое доказательство должно быть исследовано и сопоставлено с остальными добытыми по делу объективными данными. \n',
 'Оценка доказательства с точки зрения до

In [0]:
df['sents_with_jur_bez_tags'] = df['sentences_bez_jur'].apply(del_tags)


In [0]:
#def normalize_sentence(sentences):
#    lem_sent = []
#    for sentence in sentences:
#        lem_sent.append(normalize(sentence))
#    return lem_sent

In [0]:
# df['sents_with_jur_bez_tags_norm'] = df['sents_with_jur_bez_tags'].apply(normalize_sentence)

In [0]:
def split_sent(sentences):
  words = [word.strip(punct) for word in sentences.split()]
  return words  
  
df['main_part_not_lemmas'] = df['main_part_no_tags'].apply(split_sent)

In [0]:
def split_sent_with_jur_bez_tag(sentences):
  words = [word.strip() for word in sentences.split()]
  return words  

df['sents_with_jur_bez_tags_tokens'] = df['sents_with_jur_bez_tags'].apply(split_sent_with_jur_bez_tag)

In [0]:
# Обучаем FastText на tokens_main_part
import gensim
fast_text = gensim.models.FastText([text for text in df.sents_with_jur_bez_tags_tokens], size=50, min_n = 2, max_n=8)

In [0]:
from collections import Counter
import numpy as np

def get_sent_emb(text):
    
    # чтобы не доставать одно слово несколько раз
    # сделаем счетчик, а потом векторы домножим на частоту
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), 50))
    
    for i,word in enumerate(words):
        try:
            v = fast_text[word]
            vectors[i] = v*(words[word]/total) # просто умножаем вектор на частоту
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((50))
    
    return vector

In [74]:
def get_embs(text):
    embs = []
    for sent in text:
      #emb = get_sentence_embedding(sent)
      emb = get_sent_emb(sent)
      embs.append(emb)
    return embs
  
del df['sentences_embs']
df['sentences_embs'] = df['sents_with_jur_bez_tags_tokens'].apply(get_embs)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [0]:
# Функция для склеивания предложений в абзацы по cosine similarity эмбеддингов

from sklearn.metrics.pairwise import cosine_similarity

def concatenate_similar_sentences(lines, embs):
  """
  Cчитает косинусную меру сходства между 1-ым и 2-м предложением текста, 2-м и 3-м и так далее.
  Берет некоторый порог и если cosine similarity выше этого порога - объединяет предложения в один абзац.
  Возвращает список абзацев.
  """

  cur_part = lines[0]
  parts = []
  for j,i in zip(range(len(lines)-1), range(len(embs)-1)):
    sim = cosine_similarity(embs[i].reshape(-1, 1), embs[i+1].reshape(-1, 1))[0][0]
    if sim >= 0.8:
      if i != (len(embs)-1):
        cur_part = cur_part + ' ' + lines[j+1]
    else:
      parts.append(cur_part)
      cur_part = lines[j+1]
  parts.append(cur_part)
  del cur_part
  return parts
 

In [0]:
res = []
for text, emb in zip(df.sentences, df.sentences_embs):
    parts = concatenate_similar_sentences(text, emb)
    res.append(parts)

In [94]:
for elem in res[1]:
  print(elem, end='\n')

  Шериева И.Г. незаконно изготовила и хранила без цели сбыта наркотическое средство в особо крупном размере, а также содержала притон для потребления наркотических средств при следующих обстоятельствах. В дневное время в начале апреля 2011 года (точные дата и время следствием не установлены) она по месту своего прежнего жительства: ст. <адрес> <адрес>, <адрес> путем экстракции семян кондитерского мака и растворителя незаконно изготовила для личного употребления.
то есть без цели сбыта, наркотическое средство – экстракт маковой соломы, часть из которого потребила, а оставшуюся часть массой 5,27 гр. незаконно хранила для личного употребления в полимерной бутылке под навесом за кухней по указанному адресу до 16 часов 25 минут ДД.ММ.ГГГГ – до момента обнаружения и изъятия наркотического средства сотрудниками ММРО УФСКН России по КБР.
Таким образом, Шериева И.Г., нарушив ч. 1 ст. 14 Закона РФ № 3-ФЗ от 08.01.98 года «О наркотических средствах и психотропных веществах», запрещающую оборот на

In [0]:
# df['parts_emb'] = df.apply(concatenate_similar_sentences, lines=df['sentences'], embs=df['sentences_embs'])
# df['parts_emb'] = df[['sentences', 'sentences_embs']].applymap(concatenate_similar_sentences)




# норм рабочая версия apply для двух аргументов

#df['parts_emb'] = df.apply(lambda x: concatenate_similar_sentences(x.sentences, x.sentences_embs), axis=1)

In [0]:
# Функция для оценки качества деления на абзацы эмбеддингами предложений (спонсор - мишь нефедов)

def evaluate(true_pars, predicted_pars):
    assert len(true_pars) == len(predicted_pars)
    
    precisions = []
    recalls = []
    f1s = []
    
    for i in range(len(true_pars)):
        true_par = set(true_pars[i])
        predicted_par = set(predicted_pars[i])
        
        tp = len(true_par & predicted_par)
        union = len(true_par | predicted_par)
        fp = len(predicted_par - true_par)
        fn = len(true_par - predicted_par)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
          rec = tp / (tp + fn)
        
        if (prec+rec) == 0:
            f1 = 0
        else:
          f1 = (2*(prec*rec))/(prec+rec)
                    
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))

In [98]:
evaluate(df['paragraphs'], res)

Precision -  0.05
Recall -  0.09
F1 -  0.06
